# Los Pollos Hermanos

![Texto alternativo](https://th.bing.com/th/id/R100667e18919cde09899871a3487fe7e?rik=wAauP14Y%2fwSYww&riu=http%3a%2f%2fgeekyapar.com%2fwp-content%2fuploads%2f2015%2f05%2flos-pollos-hermanos-GY.jpg&ehk=EQuIJ5kFEJYnCqwkFpq3j6H5xGxrWPPglsgeWoC%2b%2fIk%3d&risl=&pid=ImgRaw)


## Introducción

Los Pollos Hermanos es una cadena de comida rápida extranjera que como parte de su expansión a Europa ha abierto el año pasado un restaurante piloto en Madrid.
Los resultados obtenidos por el restaurante han superado las expectativas de la empresa por lo que han decidido continuar su expansión. 
Para la primera fase han decidido abrir cinco restaurantes más en la ciudad de Madrid.
La dirección de la empresa no está familiarizada con la ciudad por lo que le ha encargado a una consultora, la misma que eligió la ubicación del primer restaurante, 
que busque cuales son los barrios más parecidos al actual. Ya que es donde la dirección de la empresa espera que se encuentre en mayor proporción su público objetivo. 
El propósito del estudio es determinar cuáles, de los 131 barrios de la ciudad de Madrid, son más parecidos al barrio en el que se encuentra el restaurante piloto.


 El restaurante piloto se encuentra en el barrio de Portazgo en el distrito de Puente de Vallecas

## Índice

0. Descargar e instalar librerias y paquetes

1. Cargar y preparar datos
    1.1 Descargar la tabla con los barrios y preparar el data frame
    1.2 Conseguir las coordenadas de cada barrio utilizando Nominatim de geopy.geocoders
    1.3 Obtener los datos de negocios y lugares de interes de cada barrio utilizando Foursquare
   
2. Análisis
    2.1 Codificación onehot para analizar los sitios más frecuentes en cada barrio
    2.2 Sitios más frecuentes de cada barrio
    2.3 Agrupar con K-means
    
3. Resultados
    3.1 Visualizar las agrupaciones en un mapa
    3.2 Examinar agrupaciones
    3.3 Nuevas ubicaciones propuestas
    3.4 Mapa con los barrios del cluster número 3, las posibles ubicaciones para los nuevos restaurantes de Los Pollos Hermanos
    


## 0. Descargar e instalar librerias y paquetes

In [1]:
#importar librerias

import numpy as np # librería para manejar datos vectorizados

import pandas as pd # librería para análisis de datos
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # librería para manejar archivos JSON 
!conda install -c rmg wget
!conda install -c conda-forge geopy --yes 
 
!pip install geocoder
import geocoder # import geocoder

from geopy.geocoders import Nominatim # convertir una dirección en valores de latitud y longitud

import requests # librería para manejar solicitudes
from pandas.io.json import json_normalize # librería para convertir un archivo json en un dataframe pandas

# Matplotlib y módulos asociados para graficar
import matplotlib.cm as cm
import matplotlib.colors as colors

# importar k-means desde la fase de agrupación
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # retirar el comentario de esta línea si no ha completado el laboratorio de la API de FourSquare
import folium # librería para graficar mapas 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


# 1. Cargar y preparar datos

### 1.1 Descargar la tabla con los barrios y preparar el data frame

In [91]:
# Descargar la tabla con los barrios de Madrid de Wikipedia con pandas 

url='https://es.wikipedia.org/wiki/Anexo:Barrios_administrativos_de_Madrid'


Madrid_data=pd.read_html(url)

#print(webContent[0:300])

Madrid_df=Madrid_data[0]  # Copiar la tabla a un dataframe
type(Madrid_df)
Madrid_df.head()

In [95]:
Madrid_df.shape

(131, 5)

In [96]:
#Suprimir la columna 'Imagen'
Madrid_df.drop('Imagen',1, inplace=True)
Madrid_df.head()

,Distrito,Número,Nombre,Superficie (km²)[2]​
0,Centro,11,Palacio,"1,471 km²"
1,Centro,12,Embajadores,"1,032 km²"
2,Centro,13,Cortes,"0,592 km²"
3,Centro,14,Justicia,"0,742 km²"
4,Centro,15,Universidad,"0,947 km²"


In [97]:
#Cambiar los nombres de las columnas
Madrid_df.columns= ['Distrito','Código','Barrio', 'Tamaño'] 

In [98]:

Madrid_df.head(5)



,Distrito,Código,Barrio,Tamaño
0,Centro,11,Palacio,"1,471 km²"
1,Centro,12,Embajadores,"1,032 km²"
2,Centro,13,Cortes,"0,592 km²"
3,Centro,14,Justicia,"0,742 km²"
4,Centro,15,Universidad,"0,947 km²"


In [88]:
Madrid_df.shape


(131, 4)

In [99]:
#Verificar que no haya codigos postales duplicados (si el máximo del recuento es 1, no hay codigos postales duplicados)
max(Madrid_df['Código'].value_counts())


1

In [109]:
#Convertir la columna tamaño a float, los valores no disponibles se sustituyen por la media
i=0
Tamaño=np.zeros(Madrid_df.shape[0])

for t in Madrid_df['Tamaño']:
    A=Madrid_df['Tamaño'][i]
    A=A.replace(' km²','')
    A=A.replace('n/d','0') 
    Tamaño[i]=float(A.replace(',','.'))
    i += 1
    
#Tamaño
Madrid_df['Tamaño']=Tamaño
media=Madrid_df['Tamaño'].sum()/(Madrid_df.shape[0]-(Madrid_df['Tamaño']==0).sum())
Madrid_df.iloc[Madrid_df['Tamaño']==0,3] = media

array([  1.471,   1.032,   0.592,   0.742,   0.947,   0.445,   0.967,
         1.074,   0.566,   1.396,   1.057,   0.643,   0.775,   0.75 ,
         0.64 ,   1.025,   0.49 ,   1.9  ,   0.643,   0.87 ,   0.771,
         0.852,   1.598,   0.52 ,   0.773,   1.708,   1.043,   0.762,
         1.707,   1.788,   2.16 ,   0.716,   1.193,   0.708,   0.999,
         1.163,   0.607,   0.506,   0.578,   0.611,   0.937,   0.971,
         1.068, 187.642,   1.382,   2.89 ,   1.359,   2.159,   8.977,
         6.975,  26.509,  17.47 ,   0.755,  14.248,   1.397,   3.305,
         3.554,   5.855,   1.292,   1.374,   1.679,   2.849,   9.157,
         5.419,   3.609,   0.666,   1.106,   1.899,   1.589,   1.607,
         5.613,   1.564,   1.356,   1.384,   1.463,   0.772,   0.908,
         0.775,   1.095,   5.997,   1.069,   1.724,   3.118,   1.245,
         1.849,   1.016,   0.745,   1.789,   0.999,   0.964,   0.593,
         3.198,   2.319,   0.722,   0.885,   1.052,   1.01 ,   0.559,
         0.248,   1.

In [129]:
Madrid_df.head()

,Distrito,Código,Barrio,Tamaño
0,Centro,11,Palacio,1.471
1,Centro,12,Embajadores,1.032
2,Centro,13,Cortes,0.592
3,Centro,14,Justicia,0.742
4,Centro,15,Universidad,0.947


### 1.2 Conseguir las coordenadas de cada barrio utilizando Nominatim de geopy.geocoders  

In [145]:
#Obtener las coordenadas de los barrios de Madrid

#Crear las variables donde vamos a almacernar los valores de latitud y longitud
lat=np.zeros(Madrid_df.shape[0])
long=np.zeros(Madrid_df.shape[0])

#Crear una instancia de Nominatim 
geolocator = Nominatim(user_agent="Pollos_Hermanos")
i=0

for barrio in Madrid_df['Barrio']:
    loc='{}, Madrid, España'.format(barrio)
    
    #Aplicar el método geocode para obtener las coordenadas
    location = geolocator.geocode(loc)
    if location is None:
        print(i)
    else:
        lat[i] = location.latitude
        long[i] = location.longitude
    i+=1

116
128


In [147]:
#Dos barrios no son reconocidos y devuelven un error (Valderrivas y Casco Histórico de Barajas)

Madrid_df.iloc[128,:]

Distrito                       Barajas
Código                             213
Barrio      Casco Histórico de Barajas
Tamaño                           0.609
Name: 128, dtype: object

In [152]:
Madrid_df.iloc[116,:]

Distrito      Vicálvaro
Código              193
Barrio      Valderrivas
Tamaño          4.72291
Name: 116, dtype: object

In [158]:
# Al tratarse unicamente de dos barrios se introduciran manualmente sus coordenadas
#Las coordenadas se obtienen de https://geohack.toolforge.org/geohack.php?language=es&pagename=Valderrivas&params=40.401583333333_N_-3.5990944444444_E_type:city y
# https://geohack.toolforge.org/geohack.php?language=es&pagename=Casco_Hist%C3%B3rico_de_Barajas&params=40.47361111_N_-3.57722222_E_type:city

lat[116]=40.401583
long[116]=-3.599094

lat[128]=40.473611
long[128]= -3.57722

In [160]:
#Añadimos la latitud y la longitud a Madrid_df
Madrid_df['Latitud']=lat
Madrid_df['Longitud']=long
Madrid_df.head()

,Distrito,Código,Barrio,Tamaño,Latitud,Longitud
0,Centro,11,Palacio,1.471,40.415129,-3.715618
1,Centro,12,Embajadores,1.032,40.409681,-3.701644
2,Centro,13,Cortes,0.592,40.414348,-3.698525
3,Centro,14,Justicia,0.742,40.423957,-3.695747
4,Centro,15,Universidad,0.947,40.425310,-3.706630


In [161]:
# # Obtener las coordenadas de la ciudad de Madrid con geopy 
address = 'Madrid, España'

geolocator = Nominatim(user_agent="Pollos_Hermanos")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Las coordenadas de Madrid son {}, {}.'.format(latitude, longitude))

Las coordenadas de Madrid son 40.4167047, -3.7035825.


### 1.3 Obtener los datos de negocios y lugares de interes de cada barrio utilizando Foursquare

In [ ]:
# Cargar las credenciales de Foursquare

In [162]:
CLIENT_ID = 'IP5VF5VO3F22FQKJZMKO5IS1PZFL0B1WFUIOZBDD0YRNOOKB' # su ID de Foursquare
CLIENT_SECRET = 'E4DGER2B4AVITETE2QSKABN22XWM3LTFCIKUOKPPA2KLSTWO' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

In [163]:

#Funcion para obtener los lugares que hay en una zona, a partir de los valores de latitud y longitud
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues) 

In [164]:
#Usar la función para obtener los lugares de cada distrito
Madrid_lugares = getNearbyVenues(names=Madrid_df['Barrio'], latitudes=Madrid_df['Latitud'], longitudes=Madrid_df['Longitud'])

Palacio
Embajadores
Cortes
Justicia
Universidad
Sol
Imperial
Acacias
Chopera
Legazpi
Delicias
Palos de Moguer
Atocha
Pacífico
Adelfas
Estrella
Ibiza
Jerónimos
Niño Jesús
Recoletos
Goya
Fuente del Berro
La Guindalera
Lista
Castellana
El Viso
Prosperidad
Ciudad Jardín
Hispanoamérica
Nueva España
Castilla
Bellas Vistas
Cuatro Caminos
Castillejos
Almenara
Valdeacederas
Berruguete
Gaztambide
Arapiles
Trafalgar
Almagro
Ríos Rosas
Vallehermoso
El Pardo
Fuentelarreina
Peñagrande
El Pilar
La Paz
Valverde
Mirasierra
El Goloso
Casa de Campo
Argüelles
Ciudad Universitaria
Valdezarza
Valdemarín
El Plantío
Aravaca
Los Cármenes
Puerta del Ángel
Lucero
Aluche
Campamento
Cuatro Vientos
Las Águilas
Comillas
Opañel
San Isidro
Vista Alegre
Puerta Bonita
Buenavista
Abrantes
Orcasitas
Orcasur
San Fermín
Almendrales
Moscardó
Zofío
Pradolongo
Entrevías
San Diego
Palomeras Bajas
Palomeras Sureste
Portazgo
Numancia
Pavones
Horcajo
Marroquina
Media Legua
Fontarrón
Vinateros
Ventas
Pueblo Nuevo
Quintana
Concepció

In [165]:
Madrid_lugares.shape

(3768, 7)

In [166]:
Madrid_lugares.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palacio,40.415129,-3.715618,Santa Iglesia Catedral de Santa María la Real ...,40.415767,-3.714516,Church
1,Palacio,40.415129,-3.715618,Plaza de La Almudena,40.416320,-3.713777,Plaza
2,Palacio,40.415129,-3.715618,Taberna Rayuela,40.413179,-3.713496,Tapas Restaurant
3,Palacio,40.415129,-3.715618,Corral de la Morería,40.412619,-3.714249,Performing Arts Venue
4,Palacio,40.415129,-3.715618,Palacio Real de Madrid,40.417940,-3.714259,Palace


In [167]:
#Número de categortias únicas
print('Hay {} categorias unicas.'.format(len(Madrid_lugares['Venue Category'].unique())))

Hay 274 categorias unicas.


# 2. Análisis

### 2.1 Utilizaremos la codificación onehot para analizar los sitios más frecuentes en cada barrio

In [175]:
#El primer paso es codificar los sitios por caterorias

# codificación
Madrid_onehot = pd.get_dummies(Madrid_lugares[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
Madrid_onehot['Neighborhood'] = Madrid_lugares['Neighborhood'] 


Madrid_onehot.head()

,Accessories Store,Adult Boutique,Airport,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Rental Service,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [173]:
Madrid_onehot.shape

(3768, 274)

In [271]:
# El siguiente paso es agrupar por barrios utilizando la frecuancia de la ocurrencia de cada categoría

Madrid_grouped = Madrid_onehot.groupby('Neighborhood').mean().reset_index()
Madrid_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Airport,American Restaurant,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Art Studio,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bar,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Betting Shop,Big Box Store,Bistro,Board Shop,Boarding House,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Cafeteria,Café,Camera Store,Casino,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,College Classroom,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Shop,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Embassy / Consulate,Escape Room,Ethiopian Restaurant,Event Space,Fabric Shop,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Football Stadium,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Gourmet Shop,Government Building,Greek Restaurant,Grilled Meat Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Internet Cafe,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lottery Retailer,Lounge,Market,Martial Arts School,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Motorcycle Shop,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,North Indian Restaurant,Office,Optical Shop,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoor Sculpture,Outdoors & Recreation,Paella Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pizza Place,Platform,Playground,Plaza,Pool,Portuguese Restaurant,Pub,Public Art,Ramen Restaurant,Rental Car Location,Rental Service,Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soccer Stadium,South American Restaurant,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Tennis Stadium,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Travel Lounge,Turkish Restaurant,Udon Restaurant,Used Bookstore,Vegetarian / Vegan Restaurant,Venezuelan Restaurant,Video Game Store,Vietnamese Restaurant,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,Abrantes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.1,0.0,0

In [272]:
Madrid_grouped.shape

(128, 274)

### 2.2 Sitios más frecuentes de cada barrio:

In [273]:
#Función para ordenar los sitios en orden descendente
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [274]:
#Código para obtener los 10 lugares más frecuentes en cada barrio de Madrid
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Madrid_grouped['Neighborhood']

for ind in np.arange(Madrid_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Madrid_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abrantes,Pizza Place,Restaurant,Grocery Store,Fast Food Restaurant,Bakery,Metro Station,Athletics & Sports,Gym / Fitness Center,Soccer Field,Plaza
1,Acacias,Bar,Pizza Place,Tapas Restaurant,Spanish Restaurant,Art Gallery,Park,Plaza,Event Space,Restaurant,Flea Market
2,Adelfas,Supermarket,Grocery Store,Fast Food Restaurant,Bar,Bakery,Tapas Restaurant,Soccer Field,Bookstore,Boutique,Gift Shop
3,Aeropuerto,Business Service,Football Stadium,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Yoga Studio,Dessert Shop
4,Alameda de Osuna,Tapas Restaurant,Hotel,Restaurant,Pizza Place,Smoke Shop,Shop & Service,Chinese Restaurant,Scenic Lookout,Cocktail Bar,Italian Restaurant


### 2.3 Agrupar con K-means

In [275]:
# Número de clusteres
kclusters = 7

Madrid_grouped_clustering = Madrid_grouped.drop('Neighborhood', 1)

# ejecutar k-means para obtener los clusters
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Madrid_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe (7 clusteres)
kmeans.labels_[0:10] 

array([3, 2, 2, 6, 2, 0, 2, 0, 2, 2])

In [276]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [277]:
Madrid_merged = Madrid_df


Madrid_merged =Madrid_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Barrio')

#Para 3 barrios no hay níngun sitio disponible en la base de datos de Foursquare por lo que no ha sido posible asignarles un cluster y serán eliminados

#Madrid_merged['Cluster Labels']= Madrid_merged['Cluster Labels'].apply(pd.to_numeric, errors='coerce')
Madrid_merged.dropna(axis=0,how='any', subset=['Cluster Labels'], inplace= True)
Madrid_merged.reset_index(drop=True)


Madrid_merged.head() # revisar las ultimas columnas

,Distrito,Código,Barrio,Tamaño,Latitud,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,11,Palacio,1.471,40.415129,-3.715618,2.0,Tapas Restaurant,Spanish Restaurant,Plaza,Restaurant,Bar,Historic Site,Vegetarian / Vegan Restaurant,Music Venue,Beer Bar,Mexican Restaurant
1,Centro,12,Embajadores,1.032,40.409681,-3.701644,2.0,Bar,Café,Hostel,Plaza,Spanish Restaurant,Tapas Restaurant,Bookstore,Coffee Shop,Art Gallery,Pizza Place
2,Centro,13,Cortes,0.592,40.414348,-3.698525,2.0,Hotel,Plaza,Restaurant,Bar,Spanish Restaurant,Tapas Restaurant,Theater,Mediterranean Restaurant,Café,Italian Restaurant
3,Centro,14,Justicia,0.742,40.423957,-3.695747,2.0,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Spanish Restaurant,Hotel,Deli / Bodega,Plaza,Bar,Bookstore,Café
4,Centro,15,Universidad,0.947,40.425310,-3.706630,2.0,Café,Bookstore,Cocktail Bar,Coffee Shop,Plaza,Hotel,Bar,Restaurant,Tapas Restaurant,Argentinian Restaurant


# 3. Resultados

## 3.1 Visualizar las agrupaciones en un mapa

In [ ]:
## Cada cluster de barrios se mostrará con un color diferente

In [298]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(Madrid_merged['Latitud'], Madrid_merged['Longitud'], Madrid_merged['Barrio'], Madrid_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.2 Examinar agrupaciones

In [243]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 0, Madrid_merged.columns[[0,2]+ list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Retiro,Ibiza,-3.673725,0.0,Spanish Restaurant,Tapas Restaurant,Restaurant,Italian Restaurant,Café,Seafood Restaurant,Bakery,Mediterranean Restaurant,Indian Restaurant,Movie Theater
19,Salamanca,Recoletos,-3.685743,0.0,Spanish Restaurant,Restaurant,Clothing Store,Italian Restaurant,Hotel,Japanese Restaurant,Café,Coffee Shop,Tapas Restaurant,Furniture / Home Store
20,Salamanca,Goya,-3.675843,0.0,Spanish Restaurant,Restaurant,Tapas Restaurant,Café,Bakery,Asian Restaurant,Snack Place,Seafood Restaurant,Italian Restaurant,Thai Restaurant
23,Salamanca,Lista,-3.674509,0.0,Spanish Restaurant,Restaurant,Seafood Restaurant,Indian Restaurant,Bar,Mediterranean Restaurant,Burger Joint,Hotel,Plaza,Coffee Shop
24,Salamanca,Castellana,-3.684132,0.0,Spanish Restaurant,Restaurant,Boutique,Coffee Shop,Tapas Restaurant,Japanese Restaurant,Hotel,Mediterranean Restaurant,Bistro,Bar
28,Chamartín,Hispanoamérica,-3.677202,0.0,Spanish Restaurant,Restaurant,Bar,Grocery Store,Gastropub,Japanese Restaurant,Pub,Pizza Place,Tapas Restaurant,Burger Joint
33,Tetuán,Castillejos,-3.694067,0.0,Spanish Restaurant,Restaurant,Coffee Shop,Café,Tapas Restaurant,Bar,Hotel,Cocktail Bar,Chinese Restaurant,Italian Restaurant
35,Tetuán,Valdeacederas,-3.703423,0.0,Spanish Restaurant,Art Studio,Soccer Field,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand
40,Chamberí,Almagro,-3.693044,0.0,Spanish Restaurant,Restaurant,Bar,Japanese Restaurant,Mediterranean Restaurant,Hotel,Italian Restaurant,Supermarket,French Restaurant,Café
43,Fuencarral-El Pardo,El Pardo,-3.778074,0.0,Spanish Restaurant,Restaurant,Plaza,Tapas Restaurant,Bakery,Government Building,Fried Chicken Joint,Frozen Yogurt Shop,Fish & Chips Shop,Fish Market


In [242]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 1, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
49,Fuencarral-El Pardo,Mirasierra,-3.716138,1.0,Café,Brewery,Sports Club,Gym,Convenience Store,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand
89,Moratalaz,Fontarrón,-3.646206,1.0,Brewery,Bar,Skating Rink,Café,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Furniture / Home Store,Garden


In [241]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 2, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centro,Palacio,-3.715618,2.0,Tapas Restaurant,Spanish Restaurant,Plaza,Restaurant,Bar,Historic Site,Vegetarian / Vegan Restaurant,Music Venue,Beer Bar,Mexican Restaurant
1,Centro,Embajadores,-3.701644,2.0,Bar,Café,Hostel,Plaza,Spanish Restaurant,Tapas Restaurant,Bookstore,Coffee Shop,Art Gallery,Pizza Place
2,Centro,Cortes,-3.698525,2.0,Hotel,Plaza,Restaurant,Bar,Spanish Restaurant,Tapas Restaurant,Theater,Mediterranean Restaurant,Café,Italian Restaurant
3,Centro,Justicia,-3.695747,2.0,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Spanish Restaurant,Hotel,Deli / Bodega,Plaza,Bar,Bookstore,Café
4,Centro,Universidad,-3.706630,2.0,Café,Bookstore,Cocktail Bar,Coffee Shop,Plaza,Hotel,Bar,Restaurant,Tapas Restaurant,Argentinian Restaurant
5,Centro,Sol,-3.703489,2.0,Hotel,Plaza,Spanish Restaurant,Hostel,Gourmet Shop,Cocktail Bar,Tapas Restaurant,Coffee Shop,Restaurant,Bookstore
6,Arganzuela,Imperial,-3.717322,2.0,Spanish Restaurant,Hotel,Café,Gym / Fitness Center,Monument / Landmark,Tapas Restaurant,Coffee Shop,Beer Garden,Chinese Restaurant,Grocery Store
7,Arganzuela,Acacias,-3.705957,2.0,Bar,Pizza Place,Tapas Restaurant,Spanish Restaurant,Art Gallery,Park,Plaza,Event Space,Restaurant,Flea Market
8,Arganzuela,Chopera,-3.699705,2.0,Restaurant,Italian Restaurant,Art Gallery,Spanish Restaurant,Park,Bar,Burger Joint,Coffee Shop,Tapas Restaurant,Music Venue
9,Arganzuela,Legazpi,-3.695190,2.0,Bar,Spanish Restaurant,Grocery Store,Plaza,Art Gallery,Farmers Market,Beer Garden,Restaurant,Tapas Restaurant,Argentinian Restaurant


In [240]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 3, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Fuencarral-El Pardo,Peñagrande,-3.725842,3.0,Soccer Field,Grocery Store,Argentinian Restaurant,Tapas Restaurant,Yoga Studio,Food & Drink Shop,Fish Market,Flea Market,Flower Shop,Food
60,Latina,Lucero,-3.745269,3.0,Pizza Place,Park,Snack Place,Supermarket,Fast Food Restaurant,Grocery Store,Falafel Restaurant,Gym / Fitness Center,Sandwich Place,Bar
66,Carabanchel,Opañel,-3.723178,3.0,Bar,Burger Joint,Gym / Fitness Center,Plaza,Bakery,Nightclub,Supermarket,Colombian Restaurant,Tapas Restaurant,Metro Station
68,Carabanchel,Vista Alegre,-3.740044,3.0,Grocery Store,Fast Food Restaurant,Convenience Store,Café,Breakfast Spot,Spanish Restaurant,Stadium,Sandwich Place,Tapas Restaurant,Coffee Shop
69,Carabanchel,Puerta Bonita,-3.734559,3.0,Pharmacy,Bar,Grocery Store,Fast Food Restaurant,Plaza,Pub,Stadium,Kebab Restaurant,Tapas Restaurant,Café
71,Carabanchel,Abrantes,-3.727985,3.0,Pizza Place,Restaurant,Grocery Store,Fast Food Restaurant,Bakery,Metro Station,Athletics & Sports,Gym / Fitness Center,Soccer Field,Plaza
79,Puente de Vallecas,Entrevías,-3.674761,3.0,Gym / Fitness Center,Park,Turkish Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
83,Puente de Vallecas,Portazgo,-3.648126,3.0,Bar,Tapas Restaurant,Grocery Store,Café,Pizza Place,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Stand
99,Ciudad Lineal,Costillares,-3.668512,3.0,Gym / Fitness Center,Grocery Store,Bar,Italian Restaurant,Asian Restaurant,Pet Store,Fish & Chips Shop,Music School,Tapas Restaurant,Sports Bar
101,Hortaleza,Piovera,-3.635960,3.0,Sporting Goods Shop,Sports Club,Stadium,Cheese Shop,Gym / Fitness Center,Athletics & Sports,Yoga Studio,Food & Drink Shop,Flea Market,Flower Shop


In [239]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 4, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Fuencarral-El Pardo,Fuentelarreina,-3.742602,4.0,Grocery Store,Cosmetics Shop,Hotel,Yoga Studio,Food Stand,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop
58,Latina,Los Cármenes,-3.743778,4.0,Grocery Store,Breakfast Spot,Train Station,Peruvian Restaurant,Coffee Shop,Café,Metro Station,Plaza,Hobby Shop,Ice Cream Shop
72,Usera,Orcasitas,-3.709874,4.0,Accessories Store,Kids Store,Café,Grocery Store,Sporting Goods Shop,Farmers Market,Donut Shop,Flea Market,Furniture / Home Store,Frozen Yogurt Shop
82,Puente de Vallecas,Palomeras Sureste,-3.634079,4.0,Café,Grocery Store,Gas Station,Yoga Studio,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand
108,Villaverde,Butarque,-3.674240,4.0,Grocery Store,Pharmacy,Yoga Studio,Food Stand,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Football Stadium


In [238]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 5, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
81,Puente de Vallecas,Palomeras Bajas,-3.657983,5.0,Toy / Game Store,Park,Food & Drink Shop,Bar,Frozen Yogurt Shop,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Stand
87,Moratalaz,Marroquina,-3.640568,5.0,Bar,Park,Brewery,Plaza,Nightlife Spot,Fish & Chips Shop,Garden,Furniture / Home Store,Frozen Yogurt Shop,Fried Chicken Joint
110,Villaverde,Los Ángeles,-3.699639,5.0,Bar,Supermarket,Park,Brewery,Frozen Yogurt Shop,Furniture / Home Store,Fried Chicken Joint,French Restaurant,Fountain,Fast Food Restaurant


In [237]:
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 6, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
127,Barajas,Aeropuerto,-3.574081,6.0,Business Service,Football Stadium,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Stand,Yoga Studio,Dessert Shop


## 3.3 Nuevas ubicaciones propuestas

El restaurante piloto se encuentra en el cluster número 3, por lo que se propondrán todos los barrios de este cluster como posibles ubicaciones para los nuevos restaurantes

In [279]:
# En la siguiente lista se ven todos los barrios del cluster
Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 3, Madrid_merged.columns[[0,2] + list(range(5, Madrid_merged.shape[1]))]]

,Distrito,Barrio,Longitud,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Fuencarral-El Pardo,Peñagrande,-3.725842,3.0,Soccer Field,Grocery Store,Argentinian Restaurant,Tapas Restaurant,Yoga Studio,Food & Drink Shop,Fish Market,Flea Market,Flower Shop,Food
60,Latina,Lucero,-3.745269,3.0,Pizza Place,Park,Snack Place,Supermarket,Fast Food Restaurant,Grocery Store,Falafel Restaurant,Gym / Fitness Center,Sandwich Place,Bar
66,Carabanchel,Opañel,-3.723178,3.0,Bar,Burger Joint,Gym / Fitness Center,Plaza,Bakery,Nightclub,Supermarket,Colombian Restaurant,Tapas Restaurant,Metro Station
68,Carabanchel,Vista Alegre,-3.740044,3.0,Grocery Store,Fast Food Restaurant,Convenience Store,Café,Breakfast Spot,Spanish Restaurant,Stadium,Sandwich Place,Tapas Restaurant,Coffee Shop
69,Carabanchel,Puerta Bonita,-3.734559,3.0,Pharmacy,Bar,Grocery Store,Fast Food Restaurant,Plaza,Pub,Stadium,Kebab Restaurant,Tapas Restaurant,Café
71,Carabanchel,Abrantes,-3.727985,3.0,Pizza Place,Restaurant,Grocery Store,Fast Food Restaurant,Bakery,Metro Station,Athletics & Sports,Gym / Fitness Center,Soccer Field,Plaza
79,Puente de Vallecas,Entrevías,-3.674761,3.0,Gym / Fitness Center,Park,Turkish Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Yoga Studio
83,Puente de Vallecas,Portazgo,-3.648126,3.0,Bar,Tapas Restaurant,Grocery Store,Café,Pizza Place,Fried Chicken Joint,French Restaurant,Fountain,Football Stadium,Food Stand
99,Ciudad Lineal,Costillares,-3.668512,3.0,Gym / Fitness Center,Grocery Store,Bar,Italian Restaurant,Asian Restaurant,Pet Store,Fish & Chips Shop,Music School,Tapas Restaurant,Sports Bar
101,Hortaleza,Piovera,-3.635960,3.0,Sporting Goods Shop,Sports Club,Stadium,Cheese Shop,Gym / Fitness Center,Athletics & Sports,Yoga Studio,Food & Drink Shop,Flea Market,Flower Shop


## 3.4 Mapa con los barrios del cluster número 3, las posibles ubicaciones para los nuevos restaurantes de Los Pollos Hermanos

In [291]:
# crear mapa
map_cluster3 = folium.Map(location=[latitude, longitude], zoom_start=12)

Cluster3 = Madrid_merged.loc[Madrid_merged['Cluster Labels'] == 3, ['Latitud','Longitud','Barrio','Cluster Labels']]

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(Cluster3['Latitud'], Cluster3['Longitud'], Cluster3['Barrio'], Cluster3['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_cluster3)
       
map_cluster3